In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing other required libraries
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
#from keras import Sequential laura com
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.applications.resnet import ResNet50


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation,Dropout
from tensorflow.keras.utils import to_categorical

import random
import cv2

#Import the libraries
import zipfile
import os

import pandas as pd
import matplotlib.pyplot  as plt
# from PIL import Image
from pathlib import Path
# import imagesize
import numpy as np

import cv2

import os

from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.applications import VGG19
import random

In [ ]:
def preprocessing_func(im):
    INP_SIZE = (224, 224) 
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    
#     im = cv2.imread(filename)
    im = cv2.resize(im, (INP_SIZE[0] , INP_SIZE[1]))
    im = cv2.filter2D(src=im, ddepth=-1, kernel=kernel)
    return im

In [ ]:
# ImageDataGenerator
image_generator = ImageDataGenerator(
    validation_split = 0.25,
    rescale = 1.0/255.0,
    horizontal_flip = True,
    width_shift_range =0.3,
    height_shift_range=0.3,
    preprocessing_function = preprocessing_func,
    zoom_range=0.1, 
#     shear_range = 0.2,
)

image_generator_test = ImageDataGenerator(rescale = 1.0/255.0)

In [ ]:
# paramaters
batch_size= 32
epochs=60

In [ ]:
train_dir = '../input/plant-seedlings-classification/train/'
img_size=[224,224]

train_ds = image_generator.flow_from_directory(
  train_dir,
  shuffle = True,
  target_size=(img_size[0], img_size[1]),
  class_mode = 'categorical',
  batch_size=batch_size,
  subset="training")

val_ds = image_generator.flow_from_directory(
  train_dir,
  shuffle = True,
  target_size=(img_size[0], img_size[1]),
  class_mode = 'categorical',
  batch_size=batch_size,
  subset="validation")

In [ ]:
len(train_ds.class_indices)


In [ ]:
# define model

#Initializing ResNet50
# base_model_resnet = tf.keras.applications.resnet_v2.ResNet50V2(include_top = False, weights = 'imagenet', input_shape = (224,224,3), classes = len(train_ds.class_indices))
base_model_resnet = tf.keras.applications.resnet_v2.ResNet50V2(include_top = False, weights = 'imagenet', input_shape = (224,224,3), classes = len(train_ds.class_indices))

model_resnet=Sequential()
#Add the Dense layers along with activation and batch normalization
model_resnet.add(base_model_resnet)
model_resnet.add(Flatten())
#Add the Dense layers along with activation and batch normalization
# model_resnet.add(Dense(1024,activation=('relu'),input_dim=512))
model_resnet.add(Dense(512,activation=('relu'))) 
model_resnet.add(Dropout(.4))
model_resnet.add(Dense(256,activation=('relu'))) 
model_resnet.add(Dropout(.4))
model_resnet.add(Dense(128,activation=('relu'))) 
model_resnet.add(Dropout(.4))
model_resnet.add(Dense(64,activation=('relu'))) 
model_resnet.add(Dropout(.4))

# model_resnet.add(Dense(128,activation=('relu')))
# model_resnet.add(Dropout(.2))
model_resnet.add(Dense(12,activation=('softmax')))

In [ ]:
#Compiling ResNet50
learn_rate=.001
sgd=SGD(lr=learn_rate)
model_resnet.compile(optimizer =sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# train data size / batch size
train_steps = train_ds.samples//batch_size
val_steps = val_ds.samples//batch_size

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model_resnet.fit_generator(
                train_ds,
                steps_per_epoch = train_steps,
                validation_data = val_ds, 
                validation_steps = val_steps,
                callbacks=[callback],
                epochs = epochs)

In [ ]:
f,ax=plt.subplots(2,1) #Creates 2 subplots under 1 column
#Training loss and validation loss
ax[0].plot(history.history['loss'],color='b',label='Training Loss')
ax[0].plot(history.history['val_loss'],color='r',label='Validation Loss')
#Training accuracy and validation accuracy
ax[1].plot(history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(history.history['val_accuracy'],color='r',label='Validation Accuracy')

In [ ]:
def color_segment_function(img_array):
  '''
  Generate a mask image for the images that is compatible to ImageDataGenerator
  :param image: an image read by cv2.imread
  :return result: masked image
  '''
  img_array= np.rint(img_array)
  img_array= img_array.astype('uint8')
  hsv_img= cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
  mask = cv2.inRange(hsv_img, (24, 40, 0), (60, 255, 255))
  result = cv2.bitwise_and(img_array, img_array, mask=mask)
  result= result.astype('float64')
  return result
def test_preprocessing_func(im):
    im = color_segment_function(im)
    im = preprocessing_func(im)
    return im
image_generator_test = ImageDataGenerator(rescale = 1.0/255.0, preprocessing_function = test_preprocessing_func)

In [ ]:
# model = tf.keras.models.load_model("model_resnet_224deeper")

In [ ]:
img_size = (224, 224)
test_dir = '/kaggle/input/plantt/data/'


test_generator = image_generator_test.flow_from_directory(
    test_dir,
    target_size=(img_size[0], img_size[1]),
    class_mode = None,
    batch_size=1,
    shuffle=False)

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model_resnet.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
filenames = [i.split('/')[1] for i in filenames]
results=pd.DataFrame({"file":filenames,
                      "species":predictions})
results.to_csv("./results_best.csv",index=False)

In [ ]:
results.to_csv("./submission.csv",index=False)